<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Project/Notebooks/PH_NER_Evaluate_Model_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Declare whether you are on Colab or local
colab = True

In [23]:
if colab==True:
  
  #Mount drive
  from google.colab import drive
  drive.mount('/content/drive')

  #set path to data in Google Drive
  data_path = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/All Topics"
  data_path_group = data_path[:-10]+"Topic 1/Data_Team1/" # create new data path to access files created by Team1

  #install required packages
  #!pip install duckdb
  !pip install -U sentence-transformers -q


else:
    data_path_group = "../data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     |████████████████████████████████| 85 kB 5.4 MB/s 
     |████████████████████████████████| 4.4 MB 46.5 MB/s 
     |████████████████████████████████| 1.2 MB 63.2 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
     |████████████████████████████████| 596 kB 75.0 MB/s 
     |████████████████████████████████| 6.6 MB 60.4 MB/s 


In [3]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Enhance ontology to also include parent entity id

In [4]:
#Load df of entities which were removed from the ontology during the train-val-est split
ent_removed_df = pd.read_csv(data_path_group+"NER/Data_to_split_ontology/NER_ent_to_remove_from_ontology_final.csv", index_col=0)

In [5]:
ent_removed_df.head()

,level_1,level_2,entity_id_to_remove,has_child_nodes,lvl_ent,lvl_par_ent,par_ent,ent_list,articles_list,sentences_list,num_ent,num_articles,num_sentences
0,domain specific entity,IS topic,business process outsourcing,1.0,5,4,outsourcing,"['business process outsourcing', 'knowledge pr...","[5414, 5218, 440, 5609, 5610, 5611, 1195, 2659...","['5414_1618_1639', '5218_4403_4488', '440_8401...",3,130,351
1,domain specific entity,IS topic,cybersecurity behavior,1.0,5,4,IT security category,"['privacy behavior', 'information security pol...","[1622, 1018, 3375, 2474, 6330, 5947, 1845, 425...","['1622_3154_3221', '1622_3221_3320', '1622_334...",4,121,695
2,domain specific entity,IS topic,user interface,1.0,4,3,human–computer interaction,"['user interface', 'graphical user interface',...","[1836, 434, 4244, 3373, 5413, 5415, 3196, 5606...","['1836_5404_5485', '434_1460_1510', '4244_8878...",5,980,2694
3,domain specific entity,IS topic,IT planning,1.0,4,3,information systems strategy,"['IT planning', 'IT policy', 'IT strategic pla...","[1834, 4246, 3025, 2467, 4631, 2848, 2851, 369...","['1834_10730_10748', '1834_10941_10959', '4246...",4,728,3431
4,domain specific entity,IS topic,business process management,1.0,3,2,IS topic,"['business process improvement', 'business pro...","[190, 1835, 1836, 432, 4244, 4245, 4246, 3372,...","['190_5779_5825', '190_6002_6027', '190_6702_6...",10,2148,13720


In order to determine how well our model is finding new entities within the sentences/noun phrases, we have to compare the closest entity which our model finds with the "true" parent entity id. For that we need to merge this informatin on the validation and test set. 

In [6]:
#load validation set 
ontology_test = pd.read_csv(data_path_group+"NER/NER_ontology_test.csv", index_col=0)

In [7]:
ontology_test

,entity_id,category,label,synonym
9206,sharing platform,domain specific entity,TOPIC,sharing economies
9207,sharing platform,domain specific entity,TOPIC,sharing economy
9208,sharing platform,domain specific entity,TOPIC,sharing platform
9209,sharing platform,domain specific entity,TOPIC,sharing market
9210,sharing platform,domain specific entity,TOPIC,sharing platforms
...,...,...,...,...
393,contingency table,methodological entity,METRIC,cross tabulations
394,contingency table,methodological entity,METRIC,crosstab
395,contingency table,methodological entity,METRIC,cross tables
46492,pilot survey,research method,COLLECTION_METHOD,pilot survey


In [8]:
# merge entities which were removed from train on ontology (inner merge, because ent_removed_df also includes entities which have been for the test set)
ontology_test_df = ontology_test.merge(ent_removed_df, how="inner", left_on="entity_id", right_on="entity_id_to_remove").drop(columns=["category", "label", "entity_id_to_remove"])

In [10]:
ontology_test_df.head(3)

,entity_id,synonym,level_1,level_2,has_child_nodes,lvl_ent,lvl_par_ent,par_ent,ent_list,articles_list,sentences_list,num_ent,num_articles,num_sentences
0,sharing platform,sharing economies,domain specific entity,IS topic,1.0,6,5,digital platform,"['Airbnb', 'Uber', 'sharing platform', 'ride s...","[1837, 2469, 2657, 438, 441, 5613, 5616, 5777,...","['1837_5428_5488', '2469_2695_2715', '2657_129...",6,181,1984
1,sharing platform,sharing economy,domain specific entity,IS topic,1.0,6,5,digital platform,"['Airbnb', 'Uber', 'sharing platform', 'ride s...","[1837, 2469, 2657, 438, 441, 5613, 5616, 5777,...","['1837_5428_5488', '2469_2695_2715', '2657_129...",6,181,1984
2,sharing platform,sharing platform,domain specific entity,IS topic,1.0,6,5,digital platform,"['Airbnb', 'Uber', 'sharing platform', 'ride s...","[1837, 2469, 2657, 438, 441, 5613, 5616, 5777,...","['1837_5428_5488', '2469_2695_2715', '2657_129...",6,181,1984


In [11]:
#load nounphrases val 
noun_phrases_test = pd.read_csv(data_path_group+"NER/NER_entities_test.csv", index_col=0)

Let's quickly check whether the sentences within the `noun_phrases_test` dataset are identical to the ones from the `ontology_test_df`

In [13]:
#unique number of sentences in ontology_test_df
from ast import literal_eval

# Change type from list stored as string to list
ontology_test_df["sentences_list"] = ontology_test_df.sentences_list.apply(lambda x: literal_eval(x))

In [14]:
# Create one big list of sentences
sentences_list = ontology_test_df.drop_duplicates("entity_id").sentences_list.sum(axis=0)

In [15]:
#remove duplicates
sentences_list_cln = list(set(sentences_list))
#check number of sentences
len(sentences_list_cln)

27378

In [16]:
# Compare against the number of unique sentences in the noun_phrases df
len(sentences_list_cln) == noun_phrases_test.sentence_id.unique().shape[0]

True

Perfect. Then we can continue. 

### Enhance noun phrases to also include "true" parent entity id 

In [17]:
cols = ["entity_id", "par_ent", "lvl_par_ent", "lvl_ent", "level_1", "level_2"]
noun_phrases_test_df = noun_phrases_test.merge(ontology_test_df[cols], how="left", left_on="true_ent_id", right_on="entity_id").drop(columns="entity_id").drop_duplicates()

In [18]:
noun_phrases_test_df

,article_id,sentence_id,ent_id,noun_phrases,true_ent_id,par_ent,lvl_par_ent,lvl_ent,level_1,level_2
0,190,190_7560_7615,group participant,We,0,NaN,NaN,NaN,NaN,NaN
1,190,190_7560_7615,group participant,one department,0,NaN,NaN,NaN,NaN,NaN
2,190,190_7560_7615,group participant,a hospital (cardiology,0,NaN,NaN,NaN,NaN,NaN
3,190,190_7560_7615,group participant,its related performance metrics,0,NaN,NaN,NaN,NaN,NaN
4,190,190_7560_7615,group participant,a means,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3007636,4991,4991_6118_6173,user interface,human-computer interfaces,0,NaN,NaN,NaN,NaN,NaN
3007637,4992,4992_8054_8070,group participant,The burgeoning literature,0,NaN,NaN,NaN,NaN,NaN
3007638,4992,4992_8054_8070,group participant,two groups,0,NaN,NaN,NaN,NaN,NaN
3007639,4992,4992_8054_8070,group participant,Argyris,0,NaN,NaN,NaN,NaN,NaN


Now our `noun_phrases_test_df` also includes the information about the "parent entity" of the noun phrases. We need this information to then compare the prediction from our model against the true parent entity, because ideally the model should give the parent entity id as the closest match. 

In [19]:
# Lets filter only noun phrases which have a true parent ent_id for now
noun_phrases_test_parents = noun_phrases_test_df[~noun_phrases_test_df.par_ent.isna()].copy()

In [20]:
noun_phrases_test_parents = noun_phrases_test_parents.drop_duplicates(subset="noun_phrases")
noun_phrases_test_parents.shape

(81, 10)

### Embedd noun phrases and ontology and find most similar entity per noun phrase

Now we will load the ontology_train, because this ontology does not include the entities or synonyms from the removed branches. 


In order to assess how well our model finds the correct parent entity id we will embedd the ontology train and the noun phrases, compute the cosine similarity and compare the result from prediction of our model (the closest entity to the noun phrase) and the true parent entity id. 

In [34]:
#load train set 
ontology_train = pd.read_csv(data_path_group+"NER/NER_ontology_train.csv", index_col=0)
ontology_val = pd.read_csv(data_path_group+"NER/NER_ontology_val.csv", index_col=0)
ontology_train_val = pd.concat([ontology_train, ontology_val], axis=0).reset_index(drop=True).drop_duplicates()

In [35]:
ontology_train_val.shape

(390670, 4)

In [24]:
# Import sentence_transformers 
from sentence_transformers import SentenceTransformer, util
#from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator, TripletEvaluator, LabelAccuracyEvaluator

In [25]:
model_path_syn = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/Topic 1/Data_Team1/NER/Training_results/all-MiniLM-L6-v2-Syn-2022-07-07_14-18-44"
model_path_trip = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/Topic 1/Data_Team1/NER/Training_results/all-MiniLM-L6-v2-Trip-2022-07-07_15-42-29"

In [26]:
#Load a pre-trained model & the fine tuned models
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
adapted_model_syn = SentenceTransformer(model_path_syn)
adapted_model_trip = SentenceTransformer(model_path_trip)

In [37]:
# create function based on semantic_search
def sem_search_sent_ent(model, sentences_df, entities_df, top_k = 1):


  #Compute embeddings
  embeddings_sentences = model.encode(sentences_df.noun_phrases.to_list(), convert_to_tensor=True, show_progress_bar=True)
  embeddings_entities = model.encode(entities_df.synonym.to_list(), convert_to_tensor=True, show_progress_bar=True)

  #Compute cosine-similarities for each sentence with all entities and return top k per sentence (list of list of dictonaries)
  cosine_scores = util.semantic_search(embeddings_sentences, embeddings_entities, top_k=top_k) #returns [[{'corpus_id': 483, 'score': 0.483295202255249}],...]

  #loop over results from semantic_search and create dataframe with sentence id and store in list
  dfs = []
  for idx, sim in enumerate(cosine_scores):
    test = pd.DataFrame(sim)
    test["sentence_id"] = sentences_df.iloc[idx,1]
    test["noun_phrase_txt"] = sentences_df.iloc[idx,3]
    test = test.rename(columns={"corpus_id":"entity_id"})
    dfs.append(test)

  # concat all dfs 
  dfs_df = pd.concat(dfs)
  dfs_df = dfs_df[["score", "sentence_id", "entity_id", "noun_phrase_txt"]] # reorder columns
  dfs_df[["main_entity_id"]] = dfs_df[["entity_id"]].applymap(lambda x:  entities_df.iloc[x,0]) # add "main" entity name (since we compare also with synonyms)
  dfs_df[["entity_id"]] = dfs_df[["entity_id"]].applymap(lambda x:  entities_df.iloc[x,3]) #replace entity_id_idx with real name of entity
  dfs_df = dfs_df.sort_values(by="score", ascending=False, ignore_index=True) # sort by score and reset index

  return dfs_df#, cosine_scores

Embedd noun phrases test and compare against all entities from ontology_train_val

In [38]:
##Vanilla ST
#sim_scores_vanilla = sem_search_sent_ent(model, noun_phrases_test_parents, ontology_train_val, top_k=1)
#sim_scores_vanilla.to_csv(data_path_group+"NER/similarities_test_vanilla.csv")

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/12209 [00:00<?, ?it/s]

In [39]:
##Finetuned on synonyms dataset
#sim_scores_syn = sem_search_sent_ent(adapted_model_syn, noun_phrases_test_parents, ontology_train_val, top_k=1)
#sim_scores_syn.to_csv(data_path_group+"NER/similarities_test_syn.csv")

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/12209 [00:00<?, ?it/s]

In [40]:
##Finetuned on triplets dataset
#sim_scores_trip = sem_search_sent_ent(adapted_model_trip, noun_phrases_test_parents, ontology_train_val, top_k=1)
#sim_scores_trip.to_csv(data_path_group+"NER/similarities_test_trip.csv")

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/12209 [00:00<?, ?it/s]

In [ ]:
sim_scores_vanilla = pd.read_csv(data_path_group+"NER/similarities_test_vanilla.csv", index_col=0)
sim_scores_syn = pd.read_csv(data_path_group+"NER/similarities_test_syn.csv", index_col=0)
sim_scores_trip = pd.read_csv(data_path_group+"NER/similarities_test_trip.csv", index_col=0)

In [41]:
sim_scores_vanilla.sample(3)

,score,sentence_id,entity_id,noun_phrase_txt,main_entity_id
7,0.941351,4465_12342_12358,interpretative approach,interpretive approach,interpretative method
43,0.712378,6167_7904_7921,market,sharing market,organizational participant
78,0.509612,597_2606_2651,web application,frontend,web application


### Evaluate prediction results vs. true parent entity 

Ok now lets merge on these results the information regarding the "true" parent entity id

In [42]:
def add_true_parent_entity(sim_scores_df, noun_phrases_test_parents):
    
    sim_scores_final = sim_scores_df.merge(
    noun_phrases_test_parents[['sentence_id', 'noun_phrases', 'par_ent', 'lvl_par_ent', 'lvl_ent', 'level_1', 'level_2']],\
        how="left", left_on=["sentence_id", "noun_phrase_txt"], right_on=["sentence_id", "noun_phrases"])\
        .drop(columns=["noun_phrases"])

    # compare prediction from model with true parent entity id
    sim_scores_final["correct_match_parent"] = sim_scores_final.main_entity_id==sim_scores_final.par_ent

    return sim_scores_final

In [44]:
sim_scores_vanilla_final = add_true_parent_entity(sim_scores_vanilla, noun_phrases_test_parents)
sim_scores_syn_final = add_true_parent_entity(sim_scores_syn, noun_phrases_test_parents)
sim_scores_trip_final = add_true_parent_entity(sim_scores_trip, noun_phrases_test_parents)

In [45]:
sim_scores_vanilla_final.head(3)

,score,sentence_id,entity_id,noun_phrase_txt,main_entity_id,par_ent,lvl_par_ent,lvl_ent,level_1,level_2,correct_match_parent
0,1.000000,923_9929_9942,high level languages,high level languages,programming language,information technology infrastructure,3.0,4.0,domain specific entity,IS technology,False
1,0.957111,874_421_483,interpretative techniques,interpretive techniques,interpretative method,social subjectivist paradigm,3.0,4.0,theoretical entity,research paradigm,False
2,0.951156,658_146_168,interpretative approaches,interpretive approaches,interpretative method,social subjectivist paradigm,3.0,4.0,theoretical entity,research paradigm,False


In [47]:
sim_scores_vanilla_final.to_csv(data_path_group+"NER/similarities_test_vanilla_final.csv")
sim_scores_syn_final.to_csv(data_path_group+"NER/similarities_test_syn_final.csv")
sim_scores_trip_final.to_csv(data_path_group+"NER/similarities_test_trip_final.csv")

#### Overall accuracy

Now we can measure how well the out of the shelf sentence transformer mapped the noun phrase to the correct parent entity id

In [48]:
#overall accuracy 
acc_vanilla = np.round(sim_scores_vanilla_final.correct_match_parent.sum()/sim_scores_vanilla_final.correct_match_parent.count(),2)
acc_syn = np.round(sim_scores_syn_final.correct_match_parent.sum()/sim_scores_syn_final.correct_match_parent.count(),2)
acc_trip = np.round(sim_scores_trip_final.correct_match_parent.sum()/sim_scores_trip_final.correct_match_parent.count(),2)

In [49]:
print("acc_vanilla:", acc_vanilla)
print("acc_syn:", acc_syn)
print("acc_trip:", acc_trip)

acc_vanilla: 0.09
acc_syn: 0.0
acc_trip: 0.02


#### Accuracy within similarity bins

We can also check in which ranges of cosine similarity the algorithm found the most matches

In [50]:
def create_grouped_bins(sim_scores_final):
  
  # Create bins of similarity score to check the number of correct matches compared to the labels of the database
  grouped_df = sim_scores_final.groupby(pd.cut(sim_scores_final.score, np.arange(0.,1.1,0.1)))[["correct_match_parent"]]
  corr_match_bins = np.round(grouped_df.sum()/grouped_df.count(),2).rename(columns={"correct_match_parent":"pct_correct_in_bin"})
  corr_match_bins["no_correct"]= grouped_df.sum()
  corr_match_bins["total"]= grouped_df.count()
  corr_match_bins = corr_match_bins[["total", "no_correct", "pct_correct_in_bin"]]
  corr_match_bins = corr_match_bins.iloc[::-1] #reverse order

  return corr_match_bins

In [52]:
corr_match_bins_vanilla = create_grouped_bins(sim_scores_vanilla_final)
corr_match_bins_syn = create_grouped_bins(sim_scores_syn_final)
corr_match_bins_trip = create_grouped_bins(sim_scores_trip_final)

In [53]:
corr_match_bins_vanilla

,total,no_correct,pct_correct_in_bin
score,,,
"(0.9, 1.0]",9,0,0.00
"(0.8, 0.9]",8,2,0.25
"(0.7, 0.8]",34,5,0.15
"(0.6, 0.7]",17,0,0.00
"(0.5, 0.6]",11,0,0.00
"(0.4, 0.5]",2,0,0.00
"(0.3, 0.4]",0,0,NaN
"(0.2, 0.3]",0,0,NaN
"(0.1, 0.2]",0,0,NaN


In [54]:
corr_match_bins_syn

,total,no_correct,pct_correct_in_bin
score,,,
"(0.9, 1.0]",26,0,0.0
"(0.8, 0.9]",10,0,0.0
"(0.7, 0.8]",11,0,0.0
"(0.6, 0.7]",18,0,0.0
"(0.5, 0.6]",6,0,0.0
"(0.4, 0.5]",5,0,0.0
"(0.3, 0.4]",4,0,0.0
"(0.2, 0.3]",0,0,NaN
"(0.1, 0.2]",0,0,NaN


In [55]:
corr_match_bins_trip

,total,no_correct,pct_correct_in_bin
score,,,
"(0.9, 1.0]",24,2,0.08
"(0.8, 0.9]",27,0,0.00
"(0.7, 0.8]",16,0,0.00
"(0.6, 0.7]",8,0,0.00
"(0.5, 0.6]",4,0,0.00
"(0.4, 0.5]",2,0,0.00
"(0.3, 0.4]",0,0,NaN
"(0.2, 0.3]",0,0,NaN
"(0.1, 0.2]",0,0,NaN


#### Accuracy within the different levels/ domains of the ontology

We can also check if in some levels and categories the algorithm was performing better than in others

In [ ]:
# Group matching results per level/domain and count number of correct matches and total noun phrases
matches_within_ontology_grouped = sim_scores_final.groupby(["level_1", "level_2", "lvl_ent"])\
  .aggregate(
    {'correct_match_parent' :["sum"], "noun_phrase_txt":["count"]})\
  .rename(columns={"noun_phrase_txt":"no_noun_phrases"})

In [ ]:
# Calculate accuracy across each level/ domain
matches_within_ontology_grouped[("correct_match_parent", "pct")] = np.round(
    matches_within_ontology_grouped[('correct_match_parent','sum')]/matches_within_ontology_grouped[('no_noun_phrases', 'count')]
    ,2)

In [ ]:
matches_within_ontology_grouped.head(3)

correct_match_parent  \
                                                                sum   
level_1                level_2         lvl_ent                        
domain specific entity Economic sector 3.0                        0   
                                       4.0                        0   
                                       5.0                        1   

                                               no_noun_phrases  \
                                                         count   
level_1                level_2         lvl_ent                   
domain specific entity Economic sector 3.0                   4   
                                       4.0                   2   
                                       5.0                   1   

                                               correct_match_parent  
                                                                pct  
level_1                level_2         lvl_ent                       
domain specific entity Economic sector 3.0                      0.0  
                                       4.0                      0.0  
                                       5.0                      1.0

To further enhance this grouped df we will also include the information on how many entities we originally remved from within this area of the ontology

In [ ]:
# Group number of removed number and entites per level/domain 
rem_ent_ontology_grouped = ontology_test_df.drop_duplicates(subset="entity_id")\
    .groupby(["level_1", "level_2", "lvl_ent"])\
    .aggregate({'num_ent' :["sum","count"]})

In [ ]:
rem_ent_ontology_grouped.head(3)

num_ent      
                                                   sum count
level_1                level_2         lvl_ent              
domain specific entity Economic sector 3             1     1
                                       4             2     1
                                       5             2     2

Let's now combine both grouped dataframes

In [ ]:
# Combine matching results per entity level and domain with original number of removed entities
pd.concat([matches_within_ontology_grouped, rem_ent_ontology_grouped], axis=1)

correct_match_parent  \
                                                                     sum   
level_1                level_2              lvl_ent                        
domain specific entity Economic sector      3.0                        0   
                                            4.0                        0   
                                            5.0                        1   
                       IS technology        4.0                        0   
                                            5.0                        0   
                       IS topic             3.0                        0   
                                            4.0                       21   
                                            5.0                        2   
                       study object         4.0                        0   
                                            5.0                        1   
methodological entity  data analysis method 5.0                        1   
                                            6.0                        0   
                                            7.0                        2   
                                            8.0                        0   
                                            9.0                        0   
                       research method      5.0                        0   
                                            6.0                        5   
                                            7.0                        0   
                       sampling             4.0                        0   
                       statistical metric   3.0                        0   
                       validity             4.0                        0   
                                            5.0                        0   
theoretical entity     level of analysis    4.0                        0   
                       model                3.0                        0   
                                            4.0                        0   
                                            5.0                        0   
                                            6.0                        0   
                       research paradigm    3.0                        2   
                       theory               4.0                        1   

                                                    no_noun_phrases  \
                                                              count   
level_1                level_2              lvl_ent                   
domain specific entity Economic sector      3.0                   4   
                                            4.0                   2   
                                            5.0                   1   
                       IS technology        4.0                   4   
                                            5.0                  16   
                       IS topic             3.0                   4   
                                            4.0                  28   
                                            5.0                  11   
                       study object         4.0                   2   
                                            5.0                   3   
methodological entity  data analysis method 5.0                   1   
                                            6.0                   4   
                                            7.0                   4   
                                            8.0                   1   
                                            9.0                   8   
                       research method      5.0                   2   
                                            6.0                   8   
                                            7.0                   2   
                       sampling             4.0                   1   
                       statistical metric   3.0          